<mark>프로젝트 - Movielens 📽영화🎞 추천 실습</mark>
==========================
***
***
  #### 1) 데이터 준비와 전처리
  #### 2) 분석해 봅시다.
  #### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
  #### 4) CSR matrix를 직접 만들어 봅시다.
  #### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
  #### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
  #### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
  #### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

### 1) 데이터 준비와 전처리
***

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### 평점 3점 이상만 추출

In [2]:
# 3점 이상만 남깁니다.

ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


#### ratings의 컬럼 이름을 counts로 변경

In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.

ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### 영화 데이터를 불러오기

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.

movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#### ratings와 movies 데이터프레임을 병합

In [7]:
data = pd.merge(ratings, movies)
data.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


#### 분석에 필요한 컬럼만 남기고 사용하지 않는 컬럼을 삭제

In [8]:
# 분석에 필요한 데이터만 남기고 삭제

using_cols = ['user_id', 'counts', 'title']
data = data[using_cols]
data.head(10)

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,2,5,One Flew Over the Cuckoo's Nest (1975)
2,12,4,One Flew Over the Cuckoo's Nest (1975)
3,15,4,One Flew Over the Cuckoo's Nest (1975)
4,17,5,One Flew Over the Cuckoo's Nest (1975)
5,18,4,One Flew Over the Cuckoo's Nest (1975)
6,19,5,One Flew Over the Cuckoo's Nest (1975)
7,24,5,One Flew Over the Cuckoo's Nest (1975)
8,28,3,One Flew Over the Cuckoo's Nest (1975)
9,33,5,One Flew Over the Cuckoo's Nest (1975)


#### 검색을 쉽게 하기 위해 title 컬럼을 소문자로 바꾸기

In [9]:
# title 열을 소문자로 바꿔준다.

data['title'] = data['title'].str.lower()
data.head(10)

,user_id,counts,title
0,1,5,one flew over the cuckoo's nest (1975)
1,2,5,one flew over the cuckoo's nest (1975)
2,12,4,one flew over the cuckoo's nest (1975)
3,15,4,one flew over the cuckoo's nest (1975)
4,17,5,one flew over the cuckoo's nest (1975)
5,18,4,one flew over the cuckoo's nest (1975)
6,19,5,one flew over the cuckoo's nest (1975)
7,24,5,one flew over the cuckoo's nest (1975)
8,28,3,one flew over the cuckoo's nest (1975)
9,33,5,one flew over the cuckoo's nest (1975)


### 2) 분석해 봅시다.
***
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

#### ratings에 있는 유니크한 사용자 수

In [10]:
# 유저 수
ratings['user_id'].nunique()

6039

#### ratings에 있는 유니크한 영화 개수

In [11]:
# 영화 수
movies['movie_id'].nunique()

3883

#### 가장 인기 있는 영화 30개(인기순)

In [12]:
# 인기 많은 영화

movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [13]:
# 유저별 몇 개의 영화를 봤는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
***

In [14]:
# 검증 과정을 위한 사용자 초기 정보 추가
my_favorite = ['american beauty (1999)', 
               'forrest gump (1994)', 
               'toy story (1995)',
              'jurassic park (1993)',
              'matrix, the (1999)']

# 'grint'라는 user_id가 위 영화를 5회씩 봤다고 가정
my_movie = pd.DataFrame({'user_id': ['grint']*5, 'title': my_favorite, 'counts':[5]*5})

if not data.isin({'user_id': ['grint']})['user_id'].any():
    data = data.append(my_movie)
    
data.tail(10)

,user_id,counts,title
836473,5851,5,one little indian (1973)
836474,5854,4,slaughterhouse (1987)
836475,5854,3,"promise, the (versprechen, das) (1994)"
836476,5938,4,"five wives, three secretaries and me (1998)"
836477,5948,5,identification of a woman (identificazione di ...
0,grint,5,american beauty (1999)
1,grint,5,forrest gump (1994)
2,grint,5,toy story (1995)
3,grint,5,jurassic park (1993)
4,grint,5,"matrix, the (1999)"


#### indexing 작업을 위해서 pandas.DataFrame.unique()를 이용하여 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다.

In [15]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 영화 indexing 
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [16]:
# 인덱싱이 잘 됐는지 확인
print(user_to_idx['grint'])  # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 합니다.
print(movie_to_idx['toy story (1995)'])

6039
40


In [17]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,counts,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,99
1,6039,5,160
2,6039,5,40
3,6039,5,107


### 4) CSR matrix를 직접 만들어 봅시다.
***

In [18]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
***

In [19]:
from implicit.als import AlternatingLeastSquares
import numpy as np

# Implicit AlternatingLeastSquares 모델의 선언

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [20]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)

csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [21]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [22]:
grint, toy_story = user_to_idx['grint'], movie_to_idx['toy story (1995)']
grint_vector, toy_story_vector = als_model.user_factors[grint], als_model.item_factors[toy_story]

In [23]:
grint_vector

array([ 8.3902556e-01,  2.8075713e-01, -6.2950450e-01, -8.3324260e-01,
       -1.0247378e+00,  3.8496661e-01, -3.0156221e-02,  1.2362537e+00,
       -3.3096122e-03, -1.4622669e-02,  5.7384282e-02,  5.0698125e-01,
       -2.1646243e-01, -4.8294896e-01,  2.7867708e-02,  4.6456581e-01,
       -3.2754576e-01,  4.8161852e-01,  4.2507015e-03, -3.0349828e-02,
        1.1161643e-01,  5.0918126e-01,  8.1481703e-02, -7.5027543e-01,
        3.4015503e-02, -1.9635959e-01,  4.7084719e-01,  2.3340395e-01,
        3.7158528e-01, -5.9922487e-01,  6.0551548e-01, -3.6025107e-02,
        3.9377615e-01, -5.5479750e-02,  3.8324067e-01, -2.9090506e-01,
        4.0029030e-02,  3.1947955e-01, -8.4602165e-01,  4.9062562e-01,
       -3.6030203e-01,  4.4830531e-01, -7.0187312e-01, -4.9465150e-04,
       -7.5941819e-01, -8.7352639e-01,  5.1977569e-01, -5.1258796e-01,
       -3.4402457e-01, -3.2664287e-01,  7.3498952e-01, -2.9076844e-01,
        4.0702054e-01,  3.0324993e-01,  4.2638114e-01, -6.7605209e-01,
      

In [24]:
toy_story_vector

array([-0.01211995,  0.0109512 ,  0.01074005, -0.01516482, -0.00719069,
        0.02613608, -0.02004433,  0.02819634, -0.02640413,  0.01246646,
       -0.00448715,  0.01064998, -0.01946263, -0.01324074, -0.01102937,
        0.03274656,  0.01096178,  0.01069774,  0.01986231,  0.00494709,
        0.02708036,  0.00659937, -0.01352386,  0.00349173,  0.00319658,
       -0.00305896,  0.04577035,  0.00738884,  0.0157051 , -0.00781641,
        0.01882585,  0.00083619,  0.02021739,  0.00795026, -0.02001147,
        0.01055289,  0.01444529, -0.00048977,  0.00603488,  0.04515371,
       -0.02302783,  0.01791559, -0.00010702, -0.00352085, -0.01007585,
       -0.02114804, -0.00476483, -0.01904383,  0.03684773,  0.00759067,
        0.05010354,  0.01446106, -0.00459925,  0.01458531,  0.01256415,
        0.01976478, -0.00057752,  0.02906838, -0.02457304,  0.01863265,
        0.03053399,  0.03258698, -0.0257713 , -0.0079325 ,  0.00728569,
        0.01669998,  0.01186815,  0.0093853 , -0.03464783,  0.01

In [25]:
# grint와 Toy_Story_vector를 내적하는 코드
np.dot(grint_vector, toy_story_vector)

0.5230593

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
***

In [26]:
forrest_gump = movie_to_idx['forrest gump (1994)']
forrest_gump_vector = als_model.item_factors[forrest_gump]
np.dot(grint_vector, forrest_gump_vector)

0.65979093

In [27]:
one_little_indian = movie_to_idx['one little indian (1973)']
one_little_indian_vector = als_model.item_factors[one_little_indian]
np.dot(grint_vector, one_little_indian_vector)

0.0004406448

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
***

In [28]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 

idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [29]:
get_similar_movie('toy story (1995)')

['toy story (1995)',
 'toy story 2 (1999)',
 'aladdin (1992)',
 "bug's life, a (1998)",
 'babe (1995)',
 'groundhog day (1993)',
 'lion king, the (1994)',
 'beauty and the beast (1991)',
 'pleasantville (1998)',
 "there's something about mary (1998)"]

In [30]:
get_similar_movie('forrest gump (1994)')

['forrest gump (1994)',
 'groundhog day (1993)',
 'as good as it gets (1997)',
 'clueless (1995)',
 'pretty woman (1990)',
 'four weddings and a funeral (1994)',
 'sleepless in seattle (1993)',
 'ghost (1990)',
 'dave (1993)',
 'small wonders (1996)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
***

In [31]:
user = user_to_idx['grint']

# recommend에서는 user*item CSR Matrix를 받습니다.

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(92, 0.5708084),
 (175, 0.45912185),
 (110, 0.4530189),
 (64, 0.4377558),
 (50, 0.43548074),
 (87, 0.3884011),
 (38, 0.38138747),
 (126, 0.36946228),
 (48, 0.36331555),
 (170, 0.32160303),
 (141, 0.29951936),
 (62, 0.2942831),
 (22, 0.2921079),
 (322, 0.28187263),
 (117, 0.27576318),
 (60, 0.27353325),
 (27, 0.25553194),
 (44, 0.25106737),
 (4, 0.23425196),
 (255, 0.2227214)]

In [32]:
[idx_to_movie[i[0]] for i in movie_recommended]

['terminator 2: judgment day (1991)',
 'men in black (1997)',
 'groundhog day (1993)',
 'star wars: episode vi - return of the jedi (1983)',
 'toy story 2 (1999)',
 'braveheart (1995)',
 'sixth sense, the (1999)',
 'shakespeare in love (1998)',
 'saving private ryan (1998)',
 'being john malkovich (1999)',
 'fugitive, the (1993)',
 'total recall (1990)',
 'back to the future (1985)',
 'babe (1995)',
 'star wars: episode v - the empire strikes back (1980)',
 'star wars: episode i - the phantom menace (1999)',
 'titanic (1997)',
 'star wars: episode iv - a new hope (1977)',
 "bug's life, a (1998)",
 "there's something about mary (1998)"]

In [33]:
for i in movie_recommended:
    print("{} : {:04.3f}".format(idx_to_movie[i[0]], i[1]))

terminator 2: judgment day (1991) : 0.571
men in black (1997) : 0.459
groundhog day (1993) : 0.453
star wars: episode vi - return of the jedi (1983) : 0.438
toy story 2 (1999) : 0.435
braveheart (1995) : 0.388
sixth sense, the (1999) : 0.381
shakespeare in love (1998) : 0.369
saving private ryan (1998) : 0.363
being john malkovich (1999) : 0.322
fugitive, the (1993) : 0.300
total recall (1990) : 0.294
back to the future (1985) : 0.292
babe (1995) : 0.282
star wars: episode v - the empire strikes back (1980) : 0.276
star wars: episode i - the phantom menace (1999) : 0.274
titanic (1997) : 0.256
star wars: episode iv - a new hope (1977) : 0.251
bug's life, a (1998) : 0.234
there's something about mary (1998) : 0.223


#### explain 메서드를 통해 추천에 기여한 정도를 확인

In [34]:
terminator_2 = movie_to_idx['terminator 2: judgment day (1991)']
explain = als_model.explain(user, csr_data, itemid=terminator_2)

In [35]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('matrix, the (1999)', 0.2746371574610917),
 ('jurassic park (1993)', 0.21891592794426165),
 ('toy story (1995)', 0.02941573154182588),
 ('american beauty (1999)', 0.02851221787695947),
 ('forrest gump (1994)', 0.009515127467247569)]

<kbd>회고</kbd>
=========
***
사용한 영화 데이터가 오래된 것들이라 영화 리스트를 만들때가 가장 힘들었다.<br>
다 꽤나 오래된 영화들이었고 좋아하는 영화는 없었기 때문이다.

그래도 영화 추천이라는 것을 경험해볼 수 있어서 좋았다.<br>
요즘 많은 사람들이 넷플릭스에서 영화를 추천받고, 유튜브의 알고리즘에 이끌려 동영상을 보는데 그것의 기초를 경험해볼 수 있었다.

힘내자!